### Pre-processing(Imputation)

In [104]:
import pandas as pd
import numpy as np

In [105]:
input_data = pd.read_csv('~/project/스마트농업 AI/Data/2023_smartFarm_AI_hackathon_dataset.csv')

##### 1. frmYear, frmWeek - 각 칼럼별 비율을 고려한 random imputation

In [64]:
# 결과 확인
print((input_data["frmYear"] == 0).sum())
print((input_data["frmWeek"] == 0).sum())

# 1. frmYear가 0인 값을 랜덤으로 채우기
zero_year_mask = input_data["frmYear"] == 0
num_zero_year = zero_year_mask.sum()  # frmYear가 0인 행의 개수

# 2017, 2018, 2019 년도 중에서 랜덤으로 선택하여 대체
random_years = np.random.choice([2017, 2018, 2019], size=num_zero_year)
input_data.loc[zero_year_mask, "frmYear"] = random_years

# 2. frmWeek 값을 해당 년도의 비율을 고려하여 랜덤으로 채우기
# 년도 목록 정의
years = [2017, 2018, 2019]

# 각 년도별로 처리
for year in years:
    year_mask = input_data["frmYear"] == year
    num_year = year_mask.sum()  # 해당 년도의 행의 개수
    
    if num_year > 0:
        # 해당 년도의 값이 0인 frmWeek 데이터 추출
        zero_weeks = input_data.loc[year_mask, "frmWeek"]
        zero_week_indices = zero_weeks[zero_weeks == 0].index
        
        if not zero_week_indices.empty:
            # 해당 년도의 frmWeek 비율 계산
            non_zero_weeks = zero_weeks[zero_weeks != 0]
            week_ratios = non_zero_weeks.value_counts(normalize=True)
            
            # 0인 값을 해당 년도의 비율에 맞게 랜덤으로 대체
            random_weeks = np.random.choice(week_ratios.index, size=len(zero_week_indices), p=week_ratios.values)
            
            # 대체한 값을 데이터프레임에 할당
            input_data.loc[zero_week_indices, "frmWeek"] = random_weeks


# 결과 확인
print((input_data["frmYear"] == 0).sum())
print((input_data["frmWeek"] == 0).sum())

11580
11580
0
0


##### 2. frmYear, frmWeek - date 기반 imputation

In [102]:
print((input_data["frmYear"] == 0).sum())
print((input_data["frmWeek"] == 0).sum())

# "date" 칼럼을 datetime 형식으로 변환해서 "data1"에 저장 이후 drop
input_data["date1"] = pd.to_datetime(input_data["date"], format="%Y%m%d")
input_data["date1"]

# "frmYear"와 "frmWeek" 칼럼의 값이 0인 경우를 확인하고 처리
for index, row in input_data.iterrows():
    if row["frmYear"] == 0 or row["frmWeek"] == 0:
        input_data.at[index, "frmYear"] = row["date1"].year
        input_data.at[index, "frmWeek"] = row["date1"].week

# "frmYear"와 "frmWeek" 칼럼을 정수(int) 형식으로 변환
input_data["frmYear"] = input_data["frmYear"].astype(int)
input_data["frmWeek"] = input_data["frmWeek"].astype(int)
input_data.drop("date1", axis=1, inplace=True)

# 결과 확인
print((input_data["frmYear"] == 0).sum())
print((input_data["frmWeek"] == 0).sum())

11580
11580
0
0


##### 3. inCo2(내부CO2), inTp(내부온도), inHd(내부습도), OutTp(외부온도) - frmAr(면적), frmDov(주수/평), date(날짜), frmYear(연도), frmWeek(주차)과의 KNN을 사용한 imputation

In [65]:
import pandas as pd
from sklearn.impute import KNNImputer
import numpy as np

# input_data에는 데이터프레임이 포함되어 있다고 가정합니다.

# 결과 확인
print((input_data["inCo2"] == 0).sum())
print((input_data["inTp"] == 0).sum())
print((input_data["inHd"] == 0).sum())
print((input_data["outTp"] == 0).sum())

# 사용할 칼럼 선택
selected_columns = ["frmAr", "frmDov", "date", "frmYear", "frmWeek", "inCo2", "inTp", "inHd", "outTp"]

# 0인 값을 NaN(결측치)로 변환
input_data[selected_columns] = input_data[selected_columns].replace(0, np.nan)

# KNN Imputer 객체 생성
imputer = KNNImputer(n_neighbors=5)  # n_neighbors 값은 필요에 따라 조정

# 결측치를 채울 칼럼 선택 및 결측치 처리
input_data[selected_columns] = imputer.fit_transform(input_data[selected_columns])

# 결과 확인
print((input_data["inCo2"] == 0).sum())
print((input_data["inTp"] == 0).sum())
print((input_data["inHd"] == 0).sum())
print((input_data["outTp"] == 0).sum())


12210
12210
12210
12210
0
0
0
0


##### 4. ec(급액 EC), ph(급액 PH) mean으로 imputation

In [66]:
print((input_data["ec"] == 0).sum())
print((input_data["ph"] == 0).sum())

# 대상 칼럼 선택
target_columns = ["ec", "ph"]

# 0인 값을 NaN으로 변환
input_data[target_columns] = input_data[target_columns].replace(0, np.nan)

# 각 칼럼의 평균 계산
mean_ec = input_data["ec"].mean()
mean_ph = input_data["ph"].mean()

# NaN 값을 각 칼럼의 평균 값으로 채우기
input_data["ec"].fillna(mean_ec, inplace=True)
input_data["ph"].fillna(mean_ph, inplace=True)

print((input_data["ec"] == 0).sum())
print((input_data["ph"] == 0).sum())

18360
18360
0
0


##### 5. otmsuplyqy(1회 급액량), cunt(일 급액횟수) int mode로 imputation

In [67]:
print((input_data["otmsuplyqy"] == 0).sum())
print((input_data["cunt"] == 0).sum())

# 대상 칼럼 선택
target_columns = ["otmsuplyqy", "cunt"]

for column in target_columns:
    # 0인 값을 NaN으로 변환
    input_data[column] = input_data[column].replace(0, np.nan)
    
    # NaN 값을 최빈값(mode)으로 채우기
    mode_value = input_data[column].mode().iloc[0]
    input_data[column].fillna(mode_value, inplace=True)
    
print((input_data["otmsuplyqy"] == 0).sum())
print((input_data["cunt"] == 0).sum())

21750
20280
0
0


##### 6. 